In [1]:
import pandas as pd
import os
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
df = pd.read_json("/home/jupyter/datasets/procedureevents/tokenized/procedureevents.json", orient = "records", dtype="object")

In [3]:
df.head()

,HADM_ID,ICUSTAY_ID,STORETIME,SUBJECT_ID,procedureevents
0,115071,232563,2145-03-12 23:04:00,29070,"[Blood, Cultured, 225401-1-None, Procedures, E..."
1,115071,232563,2145-03-12 23:04:00,29070,"[Urine, Culture, 225454-1-None, Procedures, El..."
2,115071,232563,2145-03-18 20:01:39,29070,"[Invasive, Ventilation, 225792-8456-hour, Vent..."
3,115071,232563,2145-03-12 23:36:00,29070,"[EKG, 225402-1-None, Procedures, Electrolytes,..."
4,115071,232563,2145-03-16 17:57:00,29070,"[PA, Catheter, 224560-5193-min, Invasive, Line..."


In [4]:
df.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258066 entries, 0 to 258065
Data columns (total 5 columns):
HADM_ID            258066 non-null object
ICUSTAY_ID         258066 non-null object
STORETIME          258066 non-null object
SUBJECT_ID         258066 non-null object
procedureevents    258066 non-null object
dtypes: object(5)
memory usage: 9.8+ MB


In [5]:
df_ICU = pd.read_csv("~/datasets/raw/ICUSTAYS.csv", dtype = "object")

In [6]:
df_ICU.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.249
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.06
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [7]:
df_ICU.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 12 columns):
ROW_ID            61532 non-null object
SUBJECT_ID        61532 non-null object
HADM_ID           61532 non-null object
ICUSTAY_ID        61532 non-null object
DBSOURCE          61532 non-null object
FIRST_CAREUNIT    61532 non-null object
LAST_CAREUNIT     61532 non-null object
FIRST_WARDID      61532 non-null object
LAST_WARDID       61532 non-null object
INTIME            61532 non-null object
OUTTIME           61522 non-null object
LOS               61522 non-null object
dtypes: object(12)
memory usage: 5.6+ MB


In [8]:
# Selecting list of HADM_ID with single ICUSTAY
df_single_ICU = df_ICU.groupby(by=['HADM_ID'], as_index=False).agg({'ICUSTAY_ID': 'count'})
df_single_ICU  = df_single_ICU[df_single_ICU['ICUSTAY_ID']==1]['HADM_ID']

In [9]:
#number of HADM_ID with single ICUSTAY
len(df_single_ICU)

54526

In [10]:
# Dropping HADM_ID with mutiple ICUSTAYS
df = df[df['HADM_ID'].isin(df_single_ICU)].reset_index(drop = True)

In [11]:
df.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222612 entries, 0 to 222611
Data columns (total 5 columns):
HADM_ID            222612 non-null object
ICUSTAY_ID         222612 non-null object
STORETIME          222612 non-null object
SUBJECT_ID         222612 non-null object
procedureevents    222612 non-null object
dtypes: object(5)
memory usage: 8.5+ MB


In [12]:
df = df.merge(df_ICU[['ICUSTAY_ID','INTIME']], how='left',left_on='ICUSTAY_ID', right_on='ICUSTAY_ID')

In [13]:
df = df.rename(columns={'INTIME':'ICU_INTIME'})

In [14]:
df.head()

,HADM_ID,ICUSTAY_ID,STORETIME,SUBJECT_ID,procedureevents,ICU_INTIME
0,115071,232563,2145-03-12 23:04:00,29070,"[Blood, Cultured, 225401-1-None, Procedures, E...",2145-03-12 20:59:55
1,115071,232563,2145-03-12 23:04:00,29070,"[Urine, Culture, 225454-1-None, Procedures, El...",2145-03-12 20:59:55
2,115071,232563,2145-03-18 20:01:39,29070,"[Invasive, Ventilation, 225792-8456-hour, Vent...",2145-03-12 20:59:55
3,115071,232563,2145-03-12 23:36:00,29070,"[EKG, 225402-1-None, Procedures, Electrolytes,...",2145-03-12 20:59:55
4,115071,232563,2145-03-16 17:57:00,29070,"[PA, Catheter, 224560-5193-min, Invasive, Line...",2145-03-12 20:59:55


In [15]:
df = df.fillna('NaN')

In [16]:
#creating a list HADM_ID of records with missing ICUSTAYID
missing_ICUSTAYID_index = df[(df['HADM_ID']!='NaN') & (df['ICUSTAY_ID']=='NaN')]['HADM_ID'].unique()

In [17]:
missing_ICUSTAYID_index

array(['190225', '102535', '113043'], dtype=object)

In [18]:
%%time
# Replace records with no ICUSTAYID with ICU INTIME associated with HADM_ID
for i in tqdm(missing_ICUSTAYID_index):
    x = df_ICU[df_ICU['HADM_ID']==i]['INTIME'].to_list()[0]
    df.loc[(df['HADM_ID']==i) & (df['ICUSTAY_ID']=='NaN'),'ICU_INTIME'] = x


CPU times: user 740 ms, sys: 0 ns, total: 740 ms
Wall time: 744 ms


In [19]:
df['STORETIME'] = pd.to_datetime(df['STORETIME'], errors='coerce')
df['ICU_INTIME'] = pd.to_datetime(df['ICU_INTIME'], errors='coerce')

In [20]:
%%time
df['icu_tdelta'] = df['STORETIME'] - df['ICU_INTIME']
df['icu_tdelta'] = df['icu_tdelta'].map(lambda x: x.total_seconds())

CPU times: user 2.54 s, sys: 48 ms, total: 2.58 s
Wall time: 2.58 s


In [21]:
df['STORETIME'] = df['STORETIME'].map(lambda x: str(x))

In [22]:
df.head()

,HADM_ID,ICUSTAY_ID,STORETIME,SUBJECT_ID,procedureevents,ICU_INTIME,icu_tdelta
0,115071,232563,2145-03-12 23:04:00,29070,"[Blood, Cultured, 225401-1-None, Procedures, E...",2145-03-12 20:59:55,7445.0
1,115071,232563,2145-03-12 23:04:00,29070,"[Urine, Culture, 225454-1-None, Procedures, El...",2145-03-12 20:59:55,7445.0
2,115071,232563,2145-03-18 20:01:39,29070,"[Invasive, Ventilation, 225792-8456-hour, Vent...",2145-03-12 20:59:55,514904.0
3,115071,232563,2145-03-12 23:36:00,29070,"[EKG, 225402-1-None, Procedures, Electrolytes,...",2145-03-12 20:59:55,9365.0
4,115071,232563,2145-03-16 17:57:00,29070,"[PA, Catheter, 224560-5193-min, Invasive, Line...",2145-03-12 20:59:55,334625.0


In [23]:
sec_24hrs = 24*60*60 #total seconds in 24 hrs
sec_48hrs = 48*60*60 #total seconds in 48 hrs

In [24]:
print("Total Records: " + str(len(df)))
print("Total Records Prior to 24hrs in ICU: " +str(len(df[df['icu_tdelta']<sec_24hrs])))
print("Total Records Prior to 48hrs in ICU: " +str(len(df[df['icu_tdelta']<sec_48hrs])))

Total Records: 222612
Total Records Prior to 24hrs in ICU: 78070
Total Records Prior to 48hrs in ICU: 124209


In [25]:
# create output path
mypath_output = "/home/jupyter/datasets/data_before_24hrs_icu/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [26]:
# extract data recorded in ans prior to first 24hrs of ICU stay
df[df['icu_tdelta']<sec_24hrs].to_json(mypath_output+"procedureevents.json", orient = 'records')

In [27]:
# create output path
mypath_output = "/home/jupyter/datasets/data_before_48hrs_icu/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [28]:
# extract data recorded in ans prior to first 48hrs of ICU stay
df[df['icu_tdelta']<sec_48hrs].to_json(mypath_output+"procedureevents.json", orient = 'records')